# Bias and Explainability

In this notebook, we will explore the concepts of bias and explainability in machine learning. We will use [the Adult Census Income public dataset](https://archive.ics.uci.edu/dataset/2/adult) (also referred to as the "Adult" or "Adult income" dataset), which is known to contain imbalances with regarding to gender and race.

Dataset Citation: Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.

## Basic setup

### Set Google Cloud resource variables

The following code will set variables specific to your Google Cloud resources that will be used in this notebook, such as the Project ID, Region, and GCS Bucket.

**Note: This notebook is intended to execute in a Vertex AI Workbench Notebook, in which case the API calls issued in this notebook are authenticated according to the permissions (e.g., service account) assigned to the Vertex AI Workbench Notebook.**

We will use the `gcloud` command to get the Project ID details from the local Google Cloud project, and assign the results to the PROJECT_ID variable. If, for any reason, PROJECT_ID is not set, you can set it manually or change it, if preferred.

We also use a default bucket name for most of the examples and activities in this book, which has the format: `{PROJECT_ID}-aiml-sa-bucket`. You can change the bucket name if preferred.

Also, we're defaulting to the **us-central1** region, but you can optionally replace this with your [preferred region](https://cloud.google.com/about/locations).

In [ ]:
PROJECT_ID_DETAILS = !gcloud config get-value project
PROJECT_ID = PROJECT_ID_DETAILS[0]  # The project ID is item 0 in the list returned by the gcloud command
BUCKET=f"{PROJECT_ID}-aiml-sa-bucket" # Optional: replace with your preferred bucket name, which must be a unique name.
REGION="us-central1" # Optional: replace with your preferred region (See: https://cloud.google.com/about/locations) 
print(f"Project ID: {PROJECT_ID}")
print(f"Bucket Name: {BUCKET}")

### Create bucket

The following code will create the bucket if it doesn't already exist.

If you get an error saying that it already exists, that's fine, you can ignore it and continue with the rest of the steps, unless you want to use a different bucket.

In [ ]:
!gsutil mb -l us-central1 gs://{BUCKET}

## Begin implementation

Now that we have performed the prerequisite steps for this activity, it's time to implement the activity.

## Bias

We start first by exploring the contents of the dataset, to explore any potential biases that may be inherently present in the dataset.

### Import libraries and data

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Define column names since the dataset doesn't have a header row
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation',
                'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']

# Load the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
adult_data = pd.read_csv(url, names=column_names, sep=r'\s*,\s*', engine='python')

adult_data.head()

### Start with a summary of the data

Get an overview of the dataset, such as the column names, the data type in each column, and the number of non-null rows for each column.

In [ ]:
adult_data.info()

View summary statistics for the data, such as the count, mean, standard deviation, min, max, and percentiles for each numeric feature.

In [ ]:
adult_data.describe()

### View distributions by gender and race

Explore whether income appears to be equally or unequally distributed by gender and race.

#### Income Distribution by Gender 

In [ ]:
plt.figure(figsize=(10, 6))
matplotlib.rcParams.update({'font.size': 15})
sns.countplot(x='income', hue='sex', data=adult_data)
plt.title('Income Distribution by Gender')
plt.show()

In the resulting graph, we can see that, overall, more people in the dataset earn more than $50,000 per year. We can also see that the numbers or males in each group far exceed the number of females in each group. This also tells us the entire dataset consists of more datapoints related to men than women. 

#### Income Distribution by Race:

In [ ]:
plt.figure(figsize=(15, 8))
sns.countplot(x='income', hue='race', data=adult_data)
plt.title('Income Distribution by Race')
plt.show()

Some important things to note from the graph outputs:

In both the "<=50K" category, and the ">50K" category, we see that the data contains many more data points for White people than any other race. This can be seen as a type of bias in the dataset. This bias may exist for multiple potential reasons, such as bias in the collection of the data, or it may happen due to other factors such as geographic location. For example, this particular dataset represents the population of a specific area, which may somewhat explain its apparent bias towards a particular race. For example, if the data were collected in Asia then it would contain many more data points for Asian people than any other race, or if it were collected in central Africa then it would contain many more data points for Black people than any other race. It's important to note any imbalances in the data and determine how they may affect the training of a machine learning model. 

Generally, if features in the dataset have much higher numbers of instances of a specific value, then an ML model's predictions will likely reflect that in some way.

In [ ]:
# Distribution of occupation by gender
plt.figure(figsize=(20, 8))
sns.countplot(y='occupation', hue='sex', data=adult_data)
plt.title('Occupational Distribution by Gender')
plt.show()

In [ ]:
# Distribution of education by race
plt.figure(figsize=(15, 8))
sns.countplot(y='education', hue='race', data=adult_data, order=adult_data['education'].value_counts().index)
plt.title('Educational Distribution by Race')
plt.show()

## Disparate Impact

The code in the following cell first creates a pivot table of the adult_data DataFrame, grouped by gender and income, with the count of people in each group as the value. Then, it adds a new column to the pivot table called rate, which is the proportion of people in each group who earn more than $50,000. Finally, it calculates the Disparate Impact (DI) by dividing the rate for females by the rate for males.

In [ ]:
# First, we'll focus on the gender-based disparity in income:

# Create a pivot table of the income outcome across genders
pivot_gender_income = adult_data.pivot_table(index='sex', columns='income', values='age', aggfunc='count')

# Calculate the favorable outcome rate for each gender
# Here, the favorable outcome is earning >50K
pivot_gender_income['rate'] = pivot_gender_income['>50K'] / (pivot_gender_income['>50K'] + pivot_gender_income['<=50K'])

# Calculate Disparate Impact (DI) as the ratio of rates between the genders
# Using 'Female' as the protected group and 'Male' as the reference group
DI = pivot_gender_income.loc['Female', 'rate'] / pivot_gender_income.loc['Male', 'rate']

print(f"Disparate Impact (Female vs Male): {DI:.3f}")

# If DI is not between 0.8 and 1.25, it might be indicative of potential bias.
if DI < 0.8 or DI > 1.25:
    print("Potential gender-based bias detected in income.")
else:
    print("No gender-based bias detected in income.")


# Explainability

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.inspection import PartialDependenceDisplay

# Load the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 
                'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
data = pd.read_csv(url, names=column_names, sep='\s*,\s*', engine='python')

# Basic preprocessing
data['income'] = data['income'].apply(lambda x: 1 if x == ">50K" else 0)
data = pd.get_dummies(data, drop_first=True)

# Drop 'fnlwgt' as it is a compound feature and not relevant for the examples in this notebook
data = data.drop('fnlwgt', axis=1)

# Separate the target from the input features
X = data.drop('income', axis=1)
y = data['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

## Feature importance

The following code looks simple, but it's achieving quite a lot. Let's break it down, line by line:

**feat_importances = pd.Series(clf.feature_importances_, index=X.columns)**

* clf.feature_importances_: This is an attribute of certain scikit-learn estimators, particularly tree-based estimators like Decision Trees, Random Forests, and Gradient Boosted Trees. It gives an array of importance scores for each feature. Higher values indicate more importance.

* X.columns: This retrieves the column names of the X DataFrame. This will be used to label each of the importance scores with the appropriate feature name.

* pd.Series(...): This constructs a pandas Series, which is a one-dimensional labeled array, using the feature importance scores as values and the column names of X as the index (labels).

Overall, this line creates a pandas Series where each item is labeled with a feature name and has a value corresponding to the importance of that feature as determined by the model.

**feat_importances.nlargest(10).plot(kind='barh')**

* feat_importances.nlargest(10): This selects the top 10 largest feature importance scores from the feat_importances Series. The resulting Series will only have 10 items, each representing one of the top 10 most important features.

* .plot(kind='barh'): This is a pandas method to plot the Series. The kind='barh' argument specifies that it should be a horizontal bar plot. Each bar represents a feature, and the length of the bar indicates the feature's importance.

To summarize all of the above, this code retrieves the feature importance scores from the model, constructs a labeled pandas Series with those scores, selects the top 10 most important features, plots their importance as a horizontal bar chart, and then displays the plot.

In [ ]:
feat_importances = pd.Series(clf.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')

### Partial Dependence Plots (PDPs)

The code in the next cell will perform the following steps:

1. Specify two feature names, 'age' and 'hours-per-week'. These are the features for which Partial Dependence Plots will be generated.
2. Generating and Displaying PDPs:

PartialDependenceDisplay.from_estimator(): 
This is a static method provided by the PartialDependenceDisplay class in the sklearn.inspection module. It is used to compute and display the partial dependence plots for the specified features.

Parameters passed to from_estimator():
* clf: This is the trained RandomForest classifier machine learning model. The method will use this model to compute the average predictions over the grid values of the features.

* X: This is the dataset on which the partial dependence is computed. The data is used to determine the unique values or grid points of the features for which the PDPs are to be plotted.

* features: The list of feature names for which PDPs are to be generated. 

Since two features are passed, the method will generate:

 - A PDP for the 'age' feature: This will show how the model's predictions change, on average, as the 'age' feature varies while other features are kept constant.
 - A PDP for the 'hours-per-week' feature: Similarly, this will show how predictions vary with changes in the 'hours-per-week' feature.
 - A 2D PDP showing interactions between 'age' and 'hours-per-week': This plot will reveal how the model's predictions change with different combinations of values for both 'age' and 'hours-per-week'.

The method will compute the partial dependence for the given features and then display the PDPs. The plots provide a visual representation of how the specified features impact the model's predictions.

In [ ]:
features = ['age', 'hours-per-week']
PartialDependenceDisplay.from_estimator(clf, X, features)

## LIME

In the following cells, I will provide a brief example for using LIME to explain individual predictions from a classifier. The original paper describing LIME can be found [here](https://arxiv.org/abs/1602.04938).

The code in the following cells will perform these steps:

1. Install LIME.
2. Import the LimeTabularExplainer class from the lime package, which is specifically designed for explaining predictions from models that operate on tabular data.
3. Create an Explainer Object, which is an instance of LimeTabularExplainer. We initialize it with the following parameters:
 - X_train.values: The training data as a NumPy array which the model has been trained on.
 - training_labels=y_train: The labels associated with the training data.
 - feature_names=X.columns.tolist(): The names of the features in the dataset, which help in making the explanation human-readable.
 - class_names=['<=50K', '>50K']: The names of the classes that the model is predicting. In this case, it's a binary classification problem with two possible outcomes: whether someone makes less than or equal to $50K, or more than $50K annually.
 - mode='classification': This indicates that the explainer is being used for a classification problem (as opposed to a regression problem).
4. Select a random instance for explanation (i = np.random.randint(0, X_test.shape[0])). This instance will be used to generate an explanation for the model's prediction.
5. Generate the explanation for the selected instance from the test set. The method clf.predict_proba is passed to the explainer, which is a function that the classifier clf uses to predict probabilities for the input data. The parameter num_features=10 tells the explainer to only use the top 10 features that are most influential for this particular prediction.
6. Displaying the explanation: The visualization will show the features that influenced the model's prediction for the selected test instance, along with their relative importance and contribution to the prediction.

In [ ]:
!pip install --quiet lime

In [ ]:
from lime import lime_tabular

explainer = lime_tabular.LimeTabularExplainer(X_train.values, training_labels=y_train, feature_names=X.columns.tolist(), 
                                              class_names=['<=50K', '>50K'], mode='classification')

i = np.random.randint(0, X_test.shape[0])
exp = explainer.explain_instance(X_test.values[i], clf.predict_proba, num_features=10)
exp.show_in_notebook()

## Counterfactuals

Next, we will use counterfactuals to understand what minimal changes would affect the model's decision-making process. The code will generate counterfactual explanations for a subset of instances from our test dataset. More specifically, for the first 100 instances in our test dataset, it will perform these steps:

1. Predict the original class.
2. Make a copy of the instance and modifies two features:
* adds 5 years to the feature at index 0 (age).
* subtracts 10 hours from the feature at index 12 ('hours-per-week').
3. Check for change in prediction: the classifier clf is used to predict the class of the perturbed instance. If the prediction is different from the original (i.e., equals the target_class), the perturbed instance is used as a counterfactual example.

In [ ]:
import warnings
from sklearn.exceptions import DataConversionWarning

warnings.filterwarnings(action='ignore', category=UserWarning)

counterfactuals = []

#for i in range(len(X_test)):
for i in range(100):
    instance = X_test.iloc[i].values
    
    # Flip the predicted class
    target_class = 1 - clf.predict([instance])
    
    # Perturb the instance
    perturbed_instance = instance.copy()
    perturbed_instance[0] += 5  # adding 5 years to age
    perturbed_instance[12] -= 10  # decreasing 10 hours from hours-per-week
    
    if clf.predict([perturbed_instance]) == target_class:
        counterfactuals.append((i, "Modified age and hours-per-week to flip the prediction"))

if counterfactuals:
    for index, msg in counterfactuals:
        print(f"For instance {index}: {msg}")
else:
    print("No counterfactuals found by modifying age and hours-per-week for any instance.")

## SHAP

For in-depth information about how the sampled Shapley method works, read the paper [Bounding the Estimation Error of Sampling-based Shapley Value Approximation](https://arxiv.org/abs/1306.4265).

In [ ]:
!pip install --quiet shap

The code in the next cell will perform the following steps:

**shap_values = shap.TreeExplainer(clf).shap_values(X_test)**

* shap.TreeExplainer(clf): Here, a [TreeExplainer](https://shap.readthedocs.io/en/latest/generated/shap.TreeExplainer.html) object from the SHAP library is being created. This specific explainer is optimized for tree-based models, such as Decision Trees, Random Forests, and Gradient Boosted Trees. The clf that we pass to it is the tree-based model that we've trained earlier in this notebook.

* .shap_values(X_test): Once the explainer is created, we use the .shap_values() method to compute the SHAP values for each sample in X_test.

SHAP values quantify the contribution of each feature to the model's prediction for a particular instance, relative to the average prediction for the entire dataset.

For a binary classification problem, the method will return a list with two arrays: one for each class. The first array is for the negative class (class 0) and the second array is for the positive class (class 1). Each array will have a shape (number of instances, number of features).

**shap.summary_plot(shap_values, X_test, plot_type="bar")** 

* shap.summary_plot(...): This function from the SHAP library creates a summary plot, combining feature importance with feature effects. Each point on the summary plot is a Shapley value for a feature and an instance. The position on the y-axis is determined by the feature and on the x-axis by the Shapley value. This means the plot provides a view of the model’s output in terms of feature importance and feature impact.

* shap_values: The SHAP values.

* X_test: This is the data for which the SHAP values are computed. It's passed to the function to map the SHAP values back to their corresponding features, making the plot interpretable.

* plot_type="bar": This argument specifies the type of summary plot. In this case, a bar plot is generated. Each bar represents a feature, and the length of the bar shows the feature's average impact on the model output. The features are sorted by their average absolute SHAP value over all the samples in X_test.

Overall, we're computing the SHAP values for the positive class on the X_test data, and then visualizing the average impact of each feature on the model's predictions using a bar plot. The longer the bar, the greater the feature's importance.

**Note: The following code can take an hour or more to complete.**

In [ ]:
import shap

shap_values = shap.TreeExplainer(clf).shap_values(X_test)
shap.summary_plot(shap_values, X_test, plot_type="bar")

Let's discuss why clf.feature_importances_ and SHAP might give different rankings for feature importance:

Computation:

Feature Importances (from clf.feature_importances_): For tree-based models, the importance of a feature is computed as the (normalized) total reduction of the criterion (like Gini impurity or mean squared error) brought by that feature. It's a cumulative metric across all the trees in the model. Features that tend to split closer to the root of trees in the ensemble typically receive higher importance.

SHAP values: SHAP values are based on cooperative game theory. The SHAP value for a feature is the average contribution of that feature value to every possible prediction (averaged over all instances). It takes into account intricate interactions with other features, as well as the feature's main effect.

Interactions:

Feature Importances: This method often overlooks feature interactions. If one feature entirely captures the information of another feature, the latter might appear as unimportant even if it's crucial in the presence of other features.

SHAP values: SHAP considers both main effects and interaction effects. It provides a more detailed view of how each feature and its interactions contribute to predictions.

Bias:

Feature Importances: The method has known biases. For instance, it might favor features with more categories or more split points. In a decision tree, high-cardinality features can lead to more splits and thus might appear artificially important.

SHAP values: SHAP values attempt to be consistent, meaning if we change the model such that it relies more on a feature, the attributed importance of that feature should not decrease. It provides a more balanced view that is less subject to biases inherent in the training process.

Global vs. Local:

Feature Importances: Provides a global perspective of feature importance averaged over the entire dataset.

SHAP values: While individual SHAP values are local (instance-specific), the summary plot gives a global perspective by aggregating over all instances. This allows capturing more complex patterns and relationships in the data.

### Get explanations from the deployed model

Conveniently, Vertex AI provides APIs and an SDK that we can use to get explanations from our models. In this section, we will use the `projects.locations.endpoints.explain` API to get explanations from the model that we deployed in our MLOps pipeline in the previous chapter. 

#### Import and initialize the Vertex AI SDK

In [ ]:
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=REGION)

#### Get the test data we created in the MLOps chapter to test our model and get explanations

First, set up constants:

In [ ]:
TEST_DATA_PREFIX = "test_data" 
TEST_DATA_DIR = f"{TEST_DATA_PREFIX}_dir"
TEST_DATA_FILE_NAME = f"{TEST_DATA_PREFIX}.jsonl"
TEST_DATASET_PATH = f"{BUCKET_URI}/{TEST_DATA_FILE_NAME}"
LOCAL_TEST_DATASET_PATH = f"./{TEST_DATA_DIR}/{TEST_DATA_FILE_NAME}"

Create a local directory to store the test data:

In [ ]:
!mkdir -p $TEST_DATA_DIR

Copy the test data to our local directory:

In [ ]:
! gsutil cp $TEST_DATASET_PATH $TEST_DATA_DIR

#### Specify endpoint details

This is the Vertex AI endpoint on which our model is hosted

In [ ]:
ENDPOINT_NAME = "mlops-endpoint"
mlops_endpoint_list = aiplatform.Endpoint.list(filter=f'display_name={ENDPOINT_NAME}', order_by='create_time desc')
new_mlops_endpoint = mlops_endpoint_list[0]
endpoint_resource_name = new_mlops_endpoint.resource_name
print(endpoint_resource_name)

#### Get explanations from the model

We will call the endpoint to get explanations for a datapoint from our test data, which we have stored in a local file. 

In this case, we're using the Sampled Shapely method which assigns credit for the outcome to each feature. This method provides a sampling approximation of exact Shapely values. Further information on the attribution methods for explanations can be found at [Overview of Explainable AI](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview).

In [ ]:
from typing import Dict

def explain_tabular_sample(
    project: str, location: str, endpoint_name: str, instance_dict: Dict
):
    endpoint = aiplatform.Endpoint(endpoint_name)

    response = endpoint.explain(instances=[instance_dict], parameters={})
    
    for explanation in response.explanations:
        print(" explanation")
        # Feature attributions.
        attributions = explanation.attributions
        for attribution in attributions:
            print("  attribution")
            print("   baseline_output_value:", attribution.baseline_output_value)
            print("   instance_output_value:", attribution.instance_output_value)
            # Convert feature_attributions to a dictionary and print
            feature_attributions_dict = dict(attribution.feature_attributions)
            print("   feature_attributions:", feature_attributions_dict)
            print("   approximation_error:", attribution.approximation_error)
            print("   output_name:", attribution.output_name)

In [ ]:
import json

with open(LOCAL_TEST_DATASET_PATH, 'r') as f:
    # Read the first line of the file
    line = f.readline()

    # Convert JSON line to Python dictionary
    instance = json.loads(line)
    
    # Convert to a list of lists (required for our model input)
    instance_list = [instance]

    # Send the inference request
    explain_tabular_sample(project=PROJECT_ID, location=REGION, endpoint_name=endpoint_resource_name, instance_dict=instance_list[0])

#### Understanding the response

Let's break down each element of the response:

**explanation:** This indicates the start of the explanation for a given instance.

**attribution:** Within each explanation, there are feature attributions. These attributions assign a value to each feature in our instance to explain how much each feature influenced the model's prediction.

**baseline_output_value:** This is the model's output value for the baseline instance. A baseline is a reference point (like an average or neutral instance) against which the prediction for your instance of interest is compared. In many explanation methods, the difference in the model's output between the instance of interest and the baseline helps understand the contributions of each feature.

**instance_output_value:** This is the model's output value for the instance we passed in for explanation. In the context of a binary classifier, this can be interpreted as the probability of the instance belonging to the positive class.

**feature_attributions_dict:**

* **'dense_input':** This is the name of the input tensor to the model. 

* **The list of numbers: [-0.1632179390639067, 0.0, ...]** represents the importance or attribution of each corresponding feature in the input for the given prediction. The length of this list matches the number of features in our model's input.

    * Each number represents the marginal contribution of that feature towards the model's prediction for the specific instance we're explaining, relative to the baseline. In essence, how much did this feature move the prediction from the average/baseline prediction?

    * Positive values indicate that the feature pushed the model's output in the positive class's direction. For binary classification, this usually means it made the model more confident in classifying the instance as the positive class.

    * Negative values indicate that the feature pushed the model's output in the negative class's direction.

    * Zero or close to zero suggests that the feature didn't have a significant impact on the prediction for this particular instance.

**approximation_error:** This is the error in the approximation used to compute the attribution values. Explanation methods typically use approximations to compute attributions. The approximation error gives an idea of the confidence we can have in the attribution values – a smaller error typically indicates more reliable attributions.

**output_name:** This is the name of the model's output tensor. 


# Lineage tracking 

The following pieces of code will list the artifacts, executions, and contexts in our GCP project.

In [ ]:
aiplatform.Artifact.list()

In [ ]:
aiplatform.Execution.list()

In [ ]:
aiplatform.Context.list()